# Stock Market Analysis
##### Shubham Mittal - CMSC320 Final Project Tutorial

### 1. Introduction
Interest in the stock market has been increasing especially after the introduction of more user friendly trading apps such as Robinhood. Financial data has also started becoming more accessible in part fueling this increase. The pandemic shook the global economy and the market experienced a lot of volatility. As a result, at the beginning of the pandemic, people started [moving away](https://www.statista.com/topics/7856/covid-19-and-investment-behavior-worldwide/#dossierKeyfigures) from the stock market to more low risk investments. However, as the pandemic has continued, a [new surge of people](https://www.cnbc.com/2021/04/08/a-large-chunk-of-the-retail-investing-crowd-got-their-start-during-the-pandemic-schwab-survey-shows.html) have started coming back to the stock market. 

Data analysis of the stock market can help traders and investors make decisions about buying and selling securities and gain an edge in the market. In this tutorial, I will analyze stock market data in different intervals to identify intraday, monthly, and yearly trends. Then we will use machine learning to also verify and predict trends. These trends could help identify periods of time that may be more lucrative for trading (a certain time of day or a certain month of a year) allowing investors to make better trading decisions. This tutorial will also help you gain a better understanding of the data science pipeline and hopefully allow you to analyze stock market data for your own purposes. 

The data science pipeline in this tutorial will consist of the following steps:
- Data Collection
- Data Cleaning & Processing
- Visualization
- Analysis & Hypothesis Testing
- Conclusions & Insights

### 2. Required Tools
In order to understand this tutorial and follow along, you will need to have a basic understanding of Python. [Click here](https://developers.google.com/edu/python/?hl=en) for a quick refresher of python. We will be using Python 3.8 in this tutorial and the following libraries: 
- [Pandas](https://pandas.pydata.org/docs/getting_started/install.html)
- [NumPy](https://numpy.org/install/)
- [yfinance](https://pypi.org/project/yfinance/) 
- [datetime](https://pypi.org/project/DateTime/)
- [pandassql](https://pypi.org/project/pandasql/)

Pandas and NumPy will be useful to manipulate and store our data. We will be getting financial data from Yahoo Finance using the yfinance library. There are also other services to obtain stock data such as the [Bloomberg Finance API](https://www.bloomberg.com/professional/product/server-api/?bbgsum-page=DG-WS-PROF-BLOG-POST-126213&tactic-page=529917) and [Quandl](https://demo.quandl.com/) and you may choose to use any of them if you prefer. 

All of the above libraries can be installed using pip (which is recommended) but the links provide more detailed information on downloading for specific OS's. Succesful execution of the cell below should import all the necessary libraries required for the rest of this tutorial. 

In [1]:
# Importing dependencies for purposes as defined above
import pandas as pd
import numpy as np
import yfinance as yfin
import datetime
import pandasql as psql

### 3. Data Collection

The first step of the data science pipeline is to obtain the data, in this case using the yfinance library. For the purposes of this tutorial, we will be analyzing two tickers, Google (GOOGL) and the S&P 500 index (^GSPC). Google is a major technological company and the S&P 500 or the Standard and Poor's 500 is an index tracking the performance of the 500 large companies listed on stock exchanges in the United States (Google being on of them).

We will be collecting data for the above two tickers in three intervals, hourly, daily, and monthly to analyze trends across different time periods. We will be collecting data from To get started, we can simply use the yfinance library's download function to obtain the necessary data. The function can take multiple parameters which can help us specify how we want the data presented. Some of the main ones are: 
- tickers (list or string): list for multiple tickers and string for a singular ticker
- start (string): start date in the format YYYY-MM-DD
- end (string): end date in the format YYYY-MM-DD
- period (string): can be used instead of start and end to get a period of the most recent data (Example: 1y)
- interval (string): specify the interval data is provided in. Valid intervals: 1m (1 minute), 1h (1 hour), 1d (1 day), 1mo (1 month)
- progress (boolean): True means a progress bar of obtaining data will be shown and False means it will not
- auto_adjust (boolean): True means data will adjust all OHLC automatically and account for things like stock split while False will not

To learn more about the options the yfinance library provides us with, [click here](https://pypi.org/project/yfinance/)

In [2]:
# Obtaining hourly, daily, and monthly data for GOOGL for the past year
hourly_GOOGL = yfin.download("GOOGL", period="365d", interval="1h", progress=False, auto_adjust=True)
daily_GOOGL = yfin.download("GOOGL", period="5y", interval="1d", progress=False, auto_adjust=True)
monthly_GOOGL = yfin.download("GOOGL", period="5y", interval="1mo", progress=False, auto_adjust=True)

# Obtaining hourly, daily, and monthly data for S&P 500 for the past year
hourly_GSPC = yfin.download("^GSPC", period="365d", interval="1h", progress=False, auto_adjust=True)
daily_GSPC = yfin.download("^GSPC", period="5y", interval="1d", progress=False, auto_adjust=True)
monthly_GSPC = yfin.download("^GSPC", period="5y", interval="1mo", progress=False, auto_adjust=True)

We have now obtained hourly data for the past year along with daily and monthly data for the past 5 years. This should be enough information for us to conduct our analysis and identify trends. Make sure to pay special attention to the values you input for parameters to ensure you are receiving data as expected.

Next, let's take a look at what the data we have collected looks like. 

In [3]:
hourly_GOOGL.head(5)

,Open,High,Low,Close,Volume
2020-11-30 09:30:00-05:00,1776.510010,1780.239990,1759.030029,1760.839844,270458
2020-11-30 10:30:00-05:00,1759.300049,1760.520020,1748.510010,1753.694946,155822
2020-11-30 11:30:00-05:00,1752.670044,1754.204956,1749.140015,1754.204956,105271
2020-11-30 12:30:00-05:00,1754.405029,1754.890015,1749.854980,1751.089966,103912
2020-11-30 13:30:00-05:00,1750.420044,1755.650024,1748.770020,1754.849976,103848


In [4]:
daily_GOOGL.head(5)

,Open,High,Low,Close,Volume
Date,,,,,
2017-05-12,957.849976,957.979980,952.059998,955.140015,1214900
2017-05-15,955.289978,962.700012,952.820007,959.219971,1337700
2017-05-16,963.549988,965.900024,960.349976,964.609985,1101500
2017-05-17,959.700012,960.989990,940.059998,942.169983,2449100
2017-05-18,943.200012,954.179993,941.270020,950.500000,1800500


In [5]:
monthly_GOOGL.head(5)

,Open,High,Low,Close,Volume
Date,,,,,
2017-06-01,990.960022,1008.609985,929.599976,929.679993,44085300
2017-07-01,933.219971,1006.190002,915.309998,945.500000,41908600
2017-08-01,947.809998,957.200012,918.599976,955.239990,32846200
2017-09-01,957.469971,975.809998,924.510010,973.719971,29626200
2017-10-01,975.650024,1063.619995,961.950012,1033.040039,36853800


From the above dataframes we can note a couple of things. All of our dataframes are indexed using the date or date and time (for the hourly dataframes). We can also notice in the daily_GOOGL dataframe, the data for 2017-05-13 and 2017-05-14 is missing. Missing values in the data obtained could be due to a number of reasons including holidays, the market closed early, or data is simply missing and was not recorded.

By looking at the data, we are able to identify different aspects that we will need to modify and process for easier analysis in further steps. All of this occurs in the second step of the data science pipeline, data cleaning and processing.

### 4. Data Cleaning & Processing
As identified above, there are a couple of steps we need to take before we will be able to analyze our data for trends. First, we will get rid of any null or missing values in our data as they will not be useful in analysis. Second, we will remove indexing by date and rename all columns for standardization of data. Third, we will label our dataframes with the hour of the day, day of the week, and month of the year for the hourly, daily, and monthly dataframes respectively. 

Once we have processed and cleaned our dataframes for easier readability and analysis, we will modify them with other features such as percent change since open to give us a better sense of how the stock prices change over time. We will also compute averages for all of the data points as with so many raw data points, visualization will not provide us with actionable information and seeing trends will be much harder. 

In [6]:
# Removing null values from our dataframes 
hourly_GOOGL = hourly_GOOGL.dropna()
daily_GOOGL = daily_GOOGL.dropna()
monthly_GOOGL = monthly_GOOGL.dropna()

hourly_GSPC = hourly_GSPC.dropna()
daily_GSPC = daily_GSPC.dropna()
monthly_GSPC = monthly_GSPC.dropna()

Next, we will resent indices and relabel the columns

In [7]:
# Resetting index to remove indexing by date
hourly_GOOGL = hourly_GOOGL.reset_index()
daily_GOOGL = daily_GOOGL.reset_index()
monthly_GOOGL = monthly_GOOGL.reset_index()

hourly_GSPC = hourly_GSPC.reset_index()
daily_GSPC = daily_GSPC.reset_index()
monthly_GSPC = monthly_GSPC.reset_index()

# Renaming Columns for standardization across dataframes
hourly_GOOGL.columns = ["Date", "Open", "High", "Low", "Close", "Volume"]
daily_GOOGL.columns = ["Date", "Open", "High", "Low", "Close", "Volume"]
monthly_GOOGL.columns = ["Date", "Open", "High", "Low", "Close", "Volume"]

hourly_GSPC.columns = ["Date", "Open", "High", "Low", "Close", "Volume"]
daily_GSPC.columns = ["Date", "Open", "High", "Low", "Close", "Volume"]
monthly_GSPC.columns = ["Date", "Open", "High", "Low", "Close", "Volume"]

Now we will define three functions that will help us label our dataframes by the hour, weekday, and month

In [8]:
# function that will label a dataframe by hour
def label_by_hour(df):
  # Will hold the labels for hours, should have values from 0 (for 9:30 am), 1 (10:30am), ..., to 6 (3:30 pm)
  hours = []
  # First hour already put into array 
  hour = 0
  hours.append(hour)
  # Start iterating from the first row of the dataframe
  for i in range(1, len(df)):
    # If we are in the same day, increment hour, otherwise rest hour
    if (df.loc[i]['Date'].weekday() == df.loc[i-1]['Date'].weekday()):
      hour += 1
      hours.append(hour)
    else:
      hour = 0
      hours.append(hour)  
      
  # Appending the hour label array to the dataframe and returning a new dataframe
  df.insert(1, "Hour", hours)
  return df

In [9]:
# function that will label a dataframe by weekday
def label_by_weekday(df):
  # Will hold the labels for days, should have values from 0 (Monday), 1 (Tuesday), ..., 4 (Friday)
  # Note, the stock market is not open on weekends
  days = []
  # Iterate through the dataframe to get the weekdays and append to the days label array
  for i in range(0, len(df)):
    days.append(df.loc[i]["Date"].weekday())
      
  # Appending the days label array to the dataframe and returning a new dataframe
  df.insert(1, "Weekday", days)
  return df
  

In [10]:
# function that will label a dataframe by month
def label_by_month(df):
  # Will hold the labels for months, should have values from 0 (January), 1 (February), ..., 12 (December)
  months = []
  # Iterate through the dataframe to get the months and append to the months label array
  for i in range(0, len(df)):
    months.append(df.loc[i]["Date"].month)
      
  # Appending the month label array to the dataframe and returning a new dataframe
  df.insert(1, "Month", months)
  return df
  

Now that we have defined functions for labelling our dataframes, we can go ahead and use them to label our stock data. After doing so, we should also verify that there were no errors while labelling our data. 

In [11]:
hourly_GOOGL = label_by_hour(hourly_GOOGL)
hourly_GSPC = label_by_hour(hourly_GSPC)

Next, let's verify that labelling by hour was appropriately done

In [12]:
print(hourly_GOOGL["Hour"].unique())
print(hourly_GSPC["Hour"].unique())

[0 1 2 3 4 5 6 7]
[0 1 2 3 4 5 6 7]


Here, for both dataframes we notice a value of 7 which should not be present as the stock market is only open for the 6 hours. To see why this may be happening, let's further examine our dataframes. 

In [13]:
hourly_GOOGL.tail(5)

,Date,Hour,Open,High,Low,Close,Volume
2545,2022-05-11 12:30:00-04:00,3,2283.570068,2295.782715,2277.070068,2282.580078,229772
2546,2022-05-11 13:30:00-04:00,4,2282.475098,2305.439941,2272.389893,2298.979980,228361
2547,2022-05-11 14:30:00-04:00,5,2301.114990,2314.539795,2282.010010,2287.429932,194230
2548,2022-05-11 15:30:00-04:00,6,2287.060059,2291.030029,2267.120117,2272.969971,273244
2549,2022-05-11 16:00:00-04:00,7,2272.050049,2272.050049,2272.050049,2272.050049,0


As we can see, the last row collected data for the end of the day at 4pm. We should also notice that the volume in that row is 0. This looks like an erroneous entry, so we will remove the row from our GOOGL dataframe before moving forward with processing our data. We will do the same for the GSPC dataframe. 

In [14]:
hourly_GOOGL = hourly_GOOGL.drop(hourly_GOOGL.index[-1])
hourly_GSPC = hourly_GSPC.drop(hourly_GSPC.index[-1])

Next, we will label our daily and monthly dataframes

In [15]:
daily_GOOGL = label_by_weekday(daily_GOOGL)
daily_GSPC = label_by_weekday(daily_GSPC)

Again, let's verify that our labelling was appropriately done. 

In [16]:
print(daily_GOOGL["Weekday"].unique())
print(daily_GSPC["Weekday"].unique())

[4 0 1 2 3]
[4 0 1 2 3]


In this case, looks like the labels were appropriately assigned. Lastly, we will label our monthly dataframes

In [17]:
monthly_GOOGL = label_by_month(monthly_GOOGL)
monthly_GSPC = label_by_month(monthly_GSPC)

Again, let's verify that the labelling was appropriately done. 

In [18]:
print(monthly_GOOGL["Month"].unique())
print(monthly_GSPC["Month"].unique())

[ 6  7  8  9 10 11 12  1  2  3  4  5]
[ 6  7  8  9 10 11 12  1  2  3  4  5]


Now that we have processed, cleaned, and labelled our dataframes, we can move forward with modifying them with other information that would be helpful in understanding the data such as percent change since open. 

In [19]:
# Calculating percent change since open for the hourly dataframes

# Arrays to hold pct_change, will be appended to dataframe
pct_change_GOOGL = []
pct_change_GSPC = []

# Providing initial values based on difference between opening and closing prices
pct_change_GOOGL.append(((hourly_GOOGL.loc[0]["Close"] - hourly_GOOGL.loc[0]["Open"])/hourly_GOOGL.loc[0]["Open"])*100)
pct_change_GSPC.append(((hourly_GSPC.loc[0]["Close"] - hourly_GSPC.loc[0]["Open"])/hourly_GSPC.loc[0]["Open"])*100)

# Calculating percent change and appending to array for GOOGL
for i in range(1, len(hourly_GOOGL)):
  change = ((hourly_GOOGL.loc[i]["Open"] - hourly_GOOGL.loc[i-1]["Open"])/hourly_GOOGL.loc[i-1]["Open"])*100
  pct_change_GOOGL.append(change)
  
# Calculating percent change and appending to array for GSPC
for i in range(1, len(hourly_GSPC)):
  change = ((hourly_GSPC.loc[i]["Open"] - hourly_GSPC.loc[i-1]["Open"])/hourly_GSPC.loc[i-1]["Open"])*100
  pct_change_GSPC.append(change)
  
hourly_GOOGL.insert(0, "Pct_change_open", pct_change_GOOGL)
hourly_GSPC.insert(0, "Pct_change_open", pct_change_GSPC)

In [20]:
# Calculating percent change since open for the daily dataframes

# Arrays to hold pct_change, will be appended to dataframe
pct_change_GOOGL = []
pct_change_GSPC = []

# Providing initial values based on difference between opening and closing prices
pct_change_GOOGL.append(((daily_GOOGL.loc[0]["Close"] - daily_GOOGL.loc[0]["Open"])/daily_GOOGL.loc[0]["Open"])*100)
pct_change_GSPC.append(((daily_GSPC.loc[0]["Close"] - daily_GSPC.loc[0]["Open"])/daily_GSPC.loc[0]["Open"])*100)

# Calculating percent change and appending to array for GOOGL
for i in range(1, len(daily_GOOGL)):
  change = ((daily_GOOGL.loc[i]["Open"] - daily_GOOGL.loc[i-1]["Open"])/daily_GOOGL.loc[i-1]["Open"])*100
  pct_change_GOOGL.append(change)
  
# Calculating percent change and appending to array for GSPC
for i in range(1, len(daily_GSPC)):
  change = ((daily_GSPC.loc[i]["Open"] - daily_GSPC.loc[i-1]["Open"])/daily_GSPC.loc[i-1]["Open"])*100
  pct_change_GSPC.append(change)
  
daily_GOOGL.insert(0, "Pct_change_open", pct_change_GOOGL)
daily_GSPC.insert(0, "Pct_change_open", pct_change_GSPC)

In [21]:
# Calculating percent change since open for the monthly dataframes

# Arrays to hold pct_change, will be appended to dataframe
pct_change_GOOGL = []
pct_change_GSPC = []

# Providing initial values based on difference between opening and closing prices
pct_change_GOOGL.append(((monthly_GOOGL.loc[0]["Close"] - monthly_GOOGL.loc[0]["Open"])/monthly_GOOGL.loc[0]["Open"])*100)
pct_change_GSPC.append(((monthly_GSPC.loc[0]["Close"] - monthly_GSPC.loc[0]["Open"])/monthly_GSPC.loc[0]["Open"])*100)

# Calculating percent change and appending to array for GOOGL
for i in range(1, len(monthly_GOOGL)):
  change = ((monthly_GOOGL.loc[i]["Open"] - monthly_GOOGL.loc[i-1]["Open"])/monthly_GOOGL.loc[i-1]["Open"])*100
  pct_change_GOOGL.append(change)
  
# Calculating percent change and appending to array for GSPC
for i in range(1, len(monthly_GSPC)):
  change = ((monthly_GSPC.loc[i]["Open"] - monthly_GSPC.loc[i-1]["Open"])/monthly_GSPC.loc[i-1]["Open"])*100
  pct_change_GSPC.append(change)
  
monthly_GOOGL.insert(0, "Pct_change_open", pct_change_GOOGL)
monthly_GSPC.insert(0, "Pct_change_open", pct_change_GSPC)

Now that we have calculated percent change since open for all of our dataframes, we can go ahead and calculate averages for all the columsn in each dataframe. We will want to group this by the labels we had created previously (Hour for hourly dataframes, Weekday for weekly dataframes, and Month for monthly dataframes).

After doing this, we should expect to see 7 groupings for the hourly dataframes (0 - 6 hours), 5 groupings for weekdays (1 - 5), and 12 groupings for months (1-12).

In [22]:
# Here, we are using the groupby method to create the groups using the Hour column and then calculating the mean for each group
avg_hourly_GOOGL = hourly_GOOGL.groupby('Hour').mean().reset_index()
avg_daily_GOOGL = daily_GOOGL.groupby('Weekday').mean()
avg_monthly_GOOGL = monthly_GOOGL.groupby('Month').mean()

avg_hourly_GSPC = hourly_GOOGL.groupby('Hour').mean()
avg_daily_GSPC = daily_GSPC.groupby('Weekday').mean()
avg_monthly_GSPC = monthly_GSPC.groupby('Month').mean()

Above we used the groupby method to help average our data. Another way to do this is by using SQL queries. I will demonstrate an example below but we will continue using our averages from above for the rest of the tutorial. 

In [31]:
# Using SELECT we are identifying the columns we would like to pick from the dataframe
# Then we are calculating the average of the columns that we want and storing then with a name using AS
# FROM is used to specify the dataframe
# GROUP BY will group our data according to the argument passed in
query = """SELECT Hour, AVG(Pct_change_open) AS Pct_change_open, AVG(Open) AS Open, AVG(High) AS High,
          AVG(Low) as Low, AVG(Close) AS Close, AVG(Volume) as Volume
          FROM hourly_GOOGL GROUP BY Hour"""
          
psql.sqldf(query, locals())

,Hour,Pct_change_open,Open,High,Low,Close,Volume
0,0,0.078405,2480.727828,2495.993387,2463.575165,2479.307211,378954.323288
1,1,-0.046587,2479.299441,2489.112114,2468.620930,2479.577758,191860.638356
2,2,0.018169,2479.696062,2487.805832,2470.926309,2479.734508,150853.452055
3,3,0.008489,2479.738733,2486.634154,2472.360385,2479.596177,125698.041096
4,4,0.003947,2480.721879,2487.869113,2473.348817,2480.630816,130146.732782
5,5,-0.000142,2480.719832,2488.209279,2473.519914,2481.294133,151899.837466
6,6,0.020722,2481.293319,2487.951881,2473.653418,2480.434807,236396.550964


Now that we have finished processing our data, we can move onto the next step of the data science pipeline to Visualization. 

### 5. Visualization
We will now plot our data and try to identify trends using both raw and averaged data. In order to plot our findings, we can use many libraries like seaborn, matplotlib, plotly, etc. Here, I will demonstrate using two of the above libraries, matplotlib and seaborn.